In [1]:
# Importing the required modules
import numpy as np # type: ignore
import pandas as pd # type: ignore

In [2]:
# Reading our dataset
dataset = pd.read_csv('crop_yield.csv')

In [3]:
dataset.head()

,Crop,Crop_Year,Season,State,Area,Production,Annual_Rainfall,Fertilizer,Pesticide,Yield
0,Arecanut,1997,Whole Year,Assam,73814.0,56708,2051.4,7024878.38,22882.34,0.796087
1,Arhar/Tur,1997,Kharif,Assam,6637.0,4685,2051.4,631643.29,2057.47,0.710435
2,Castor seed,1997,Kharif,Assam,796.0,22,2051.4,75755.32,246.76,0.238333
3,Coconut,1997,Whole Year,Assam,19656.0,126905000,2051.4,1870661.52,6093.36,5238.051739
4,Cotton(lint),1997,Kharif,Assam,1739.0,794,2051.4,165500.63,539.09,0.420909


In [4]:
# Getting information about our data
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19689 entries, 0 to 19688
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Crop             19689 non-null  object 
 1   Crop_Year        19689 non-null  int64  
 2   Season           19689 non-null  object 
 3   State            19689 non-null  object 
 4   Area             19689 non-null  float64
 5   Production       19689 non-null  int64  
 6   Annual_Rainfall  19689 non-null  float64
 7   Fertilizer       19689 non-null  float64
 8   Pesticide        19689 non-null  float64
 9   Yield            19689 non-null  float64
dtypes: float64(5), int64(2), object(3)
memory usage: 1.5+ MB


In [5]:
# Statistical description of data
dataset.describe()

,Crop_Year,Area,Production,Annual_Rainfall,Fertilizer,Pesticide,Yield
count,19689.000000,1.968900e+04,1.968900e+04,19689.000000,1.968900e+04,1.968900e+04,19689.000000
mean,2009.127584,1.799266e+05,1.643594e+07,1437.755177,2.410331e+07,4.884835e+04,79.954009
std,6.498099,7.328287e+05,2.630568e+08,816.909589,9.494600e+07,2.132874e+05,878.306193
min,1997.000000,5.000000e-01,0.000000e+00,301.300000,5.417000e+01,9.000000e-02,0.000000
25%,2004.000000,1.390000e+03,1.393000e+03,940.700000,1.880146e+05,3.567000e+02,0.600000
50%,2010.000000,9.317000e+03,1.380400e+04,1247.600000,1.234957e+06,2.421900e+03,1.030000
75%,2015.000000,7.511200e+04,1.227180e+05,1643.700000,1.000385e+07,2.004170e+04,2.388889
max,2020.000000,5.080810e+07,6.326000e+09,6552.700000,4.835407e+09,1.575051e+07,21105.000000


In [6]:
# Function to cap outliers only for numeric columns
def cap_outliers(dataset, column):
    if pd.api.types.is_numeric_dtype(dataset[column]):
        lower_bound = dataset[column].quantile(0.01)
        upper_bound = dataset[column].quantile(0.99)
        dataset[column] = dataset[column].clip(lower=lower_bound, upper=upper_bound)

# Apply the function to numeric columns only
for column in dataset.columns:
    cap_outliers(dataset, column)

In [7]:
dataset.describe()

,Crop_Year,Area,Production,Annual_Rainfall,Fertilizer,Pesticide,Yield
count,19689.000000,1.968900e+04,1.968900e+04,19689.000000,1.968900e+04,19689.000000,19689.000000
mean,2009.125705,1.599518e+05,9.067774e+05,1428.350138,2.124646e+07,42382.254291,5.107530
std,6.495098,4.796407e+05,4.381942e+06,769.742365,6.401437e+07,127797.071394,14.591018
min,1997.000000,3.000000e+00,2.000000e+00,345.600000,4.002000e+02,0.720000,0.081241
25%,2004.000000,1.390000e+03,1.393000e+03,940.700000,1.880146e+05,356.700000,0.600000
50%,2010.000000,9.317000e+03,1.380400e+04,1247.600000,1.234957e+06,2421.900000,1.030000
75%,2015.000000,7.511200e+04,1.227180e+05,1643.700000,1.000385e+07,20041.700000,2.388889
max,2019.000000,3.299226e+06,3.757112e+07,4472.300000,4.427270e+08,885873.261600,104.273387


In [8]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score


# Encoding categorical variables (e.g., one-hot encoding)
df_encoded = pd.get_dummies(dataset, columns=['Crop', 'Season', 'State'])

# Split data into features (X) and target (y)
X = df_encoded.drop(columns=['Yield', 'Crop_Year'])
y = df_encoded['Yield']

# Train-Test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Instantiate and fit Random Forest Regressor
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'Mean Squared Error: {mse}')
print(r2)

Mean Squared Error: 4.4879235051629065
0.9782026029175848


In [9]:
import joblib

# Save the trained model to a pickle file
model_filename = 'random_forest_model.pkl'
joblib.dump(model, model_filename)

['random_forest_model.pkl']

In [ ]:
# Load the trained model
model = joblib.load('random_forest_model.pkl')

# List of encoded columns used in the trained model (you can get this from your training process)
encoded_columns = model.feature_names_in_  # if you have used scikit-learn version 1.0 or higher

# Function to preprocess the input data
def preprocess_input(crop, season, state, area, production, annual_rainfall, fertilizer, pesticide):
    # Create a dictionary from the input data
    input_data = {
        'Crop': [crop],
        'Season': [season],
        'State': [state],
        'Area': [area],
        'Production': [production],
        'Annual_Rainfall': [annual_rainfall],
        'Fertilizer': [fertilizer],
        'Pesticide': [pesticide]
    }

    # Convert input data to DataFrame
    input_df = pd.DataFrame(input_data)

    # Apply the same pd.get_dummies() transformation as in the original dataset
    input_df_encoded = pd.get_dummies(input_df, columns=['Crop', 'Season', 'State'])

    # Add any missing columns with a value of 0 (in case the input doesn't have all categories)
    for col in encoded_columns:
        if col not in input_df_encoded.columns:
            input_df_encoded[col] = 0

    # Reorder columns to match the training data columns
    input_df_encoded = input_df_encoded[encoded_columns]

    return input_df_encoded

# Input features
crop = input("Enter the crop type: ")
season = input("Enter the season: ")
state = input("Enter the state: ")
area = float(input("Enter the area (in hectares): "))
production = float(input("Enter the amount of production (in metric tonnes):"))
annual_rainfall = float(input("Enter the annual rainfall (in mm): "))
fertilizer = float(input("Enter the amount of fertilizer (in kg): "))
pesticide = float(input("Enter the amount of pesticide (in kg): "))

# Preprocess input data
input_data_encoded = preprocess_input(crop, season, state, area, production, annual_rainfall, fertilizer, pesticide)

# Predict the yield using the pre-trained model
predicted_yield = model.predict(input_data_encoded)

print(f"Predicted yield: {predicted_yield}")